In [ ]:
!pip install openai
!pip install tiktoken

In [ ]:
import openai
from getpass import getpass
key = getpass('Enter Deepseek API key: ')


In [ ]:
from openai import OpenAI

client = OpenAI(api_key=key, base_url="https://api.deepseek.com")

In [ ]:
import urllib.request
import tiktoken
from bs4 import BeautifulSoup
choice = input("Please provide the material to build the interview from. \nEnter '1' to input a URL or '2' to input plain text: ")
if choice == '1':
  encoding = tiktoken.get_encoding("cl100k_base")
  url = input("Enter the URL: ")
  response = urllib.request.urlopen(url).read()
  soup = BeautifulSoup(response, features ="html.parser")
  input_text = soup.get_text()
elif choice == '2':
  input_text = input("Enter the plain text: ")

In [111]:
def generate_with_reasoner(input_text, prompt):
  response = client.chat.completions.create(
      model="deepseek-reasoner",
      messages=[
          {"role": "system", "content": prompt},
          {"role": "user", "content": input_text},
      ],
      temperature=0,
      frequency_penalty=0,
      presence_penalty=0,
  )
  return response

Prompt for identifying conditions and constructing Docassemble questions/events. Output will be saved as skeleton.yml.

In [112]:
prompt_1 = """You are to assume the role of a legal expert building an expert system. The input text describes some
            criteria for a legal conclusion being either true or false. You will return a set of Docassemble questions
            in YAML format.

            To generate questions, you must identify all conditions that can affect the conclusion. Some will
            have a direct effect, and some may have an overriding effect. A condition that has an overriding effect
            invalidates the necessity of another condition. When identifying explicit conditions, carefully consider
            whether they are independent or dependent on other information.

            There should only be two events - one which occurs when the conclusion is true and
            one which occurs when the conclusion is false.
            For each event:

            1. Define the "event" name.
            2. Define the event "question" (the written conclusion that the user will see).

            There must be a number of questions necessary to accurately determine the conclusion.
            For all questions:

            1. Define a clear "question" for the user.
            2. Define the question type: either "yesno" or "date".
            3. Define the field name which will store the user's response (boolean or date value).

            Rules for the output:
            - The question type must be either "yesno" or "date". No other types are permitted.
            - Questions that relate to a specific time period or date must use a "date" field.
            - Do not include unnecessary or duplicate questions.
            - Do not include additional information that describes eligibility cutoffs, such as:
                - “Required after [date]”
                - “Does not apply if...”.
              These types of conditions will be handled by interview logic when the system is built.
            - If a legal condition requires checking exceptions (e.g. unless, except), create an explicit question to capture that.
            - Questions must straightforward to answer, if a question is too complex, separate into multiple questions.
            - Do not define any interview logic or code blocks, as this will be implemented after.


            Example Input:

            "Check if your family members can get pre-settled status or settled status. If you’re an EU, EEA or Swiss citizen living in the UK,
            some of your family can also apply to come and live in the UK. They can apply for pre-settled or settled status from the EU Settlement
            Scheme if both:
            - you have pre-settled or settled status
            - your relationship with your family member started by 31 December 2020 - unless you’re a Swiss citizen
            If your family member is a child who was born after 31 December 2020, you can also apply for them to come and live in the UK.
            If you came to the UK on a visa after 31 December 2020, you can't use the EU Settlement Scheme to bring your family to the UK.
            You’ll need to check if your visa allows you to bring your family to the UK. The EEA includes EU countries and also Iceland, Liechtenstein
            and Norway."

            Example Output:
            ---
            question: |
              Are you an EU citizen?
            yesno: is_eu_citizen
            ---
            question: |
              Do you have settled status?
            yesno: is_settled
            ---
            question: |
              Do you have pre-settled status?
            yesno: is_pre_settled
            ---
            question: |
              What date did your relationship with your family member start?
            fields:
              - Date: date_of_relation_start
                datatype: date
            ---
            question: |
              Are you Swiss?
            yesno: is_swiss
            ---
            question: |
              What is your family member's date of birth?
            fields:
              - Date: date_of_birth
                datatype: date
            ---
            question: |
              Did you come to the UK on a visa?
            yesno: arrived_on_visa
            ---
            question: |
              What date did you arrive on this visa?
            fields:
              - Date: date_of_arrival
                datatype: date
            ---
            event: cannot_apply_event
            question: |
              You do not meet the basic requirements for your family to apply for pre-settled or settled status from the EU settlement scheme.
            buttons:
            - Exit: exit
            ---
            event: can_apply_event
            question: |
              You meet the basic requirements for your family to apply for pre-settled or settled status from the EU settlement scheme.
            buttons:
            - Exit: exit
            ---

         """

In [113]:
response = generate_with_reasoner(input_text, prompt_1)
reasoning_content = response.choices[0].message.reasoning_content
skeleton = response.choices[0].message.content

Prompt for adapting skeleton.yml to include interview logic, based on relationships between conditions. Output will be saved as interview.yml

In [114]:
prompt_2 = """You are assuming the role of a legal expert developing an expert system. Your task is to complete a docassemble interview that reaches a legal conclusion.

            You will be provided with:
            - A YAML file containing Docassemble questions
            - Legal material

            You must generate the complete interview, including all questions and code blocks.

            Use only the legal text to determine the logic and construct the necessary questions.

            Code Block Requirements:

            - Code blocks must adhere to the structure provided in the example. When writing logic, use a single,
              cohesive boolean expression with proper logical operators and parentheses.
              Do not use multiple "if" statements.

            - The first code block must be the only block marked "mandatory: True".
              It must contain the main logic that evaluates to a boolean value (e.g., "can_apply = True" or "False").
              This block must trigger an event depending on the outcome of the evaluation ("can_apply_event" or "cannot_apply_event").

            - Subsequent blocks may define boolean variables based on conditions (e.g. date comparisons or grouped expressions).
              These blocks should appear after the main logic so they are only be evaluated if needed.
              Do not give booleans default values at the start of a code block, as this may cause the code block to be terminated
              without completing.

            - Use the "as_datetime()" function from docassemble's "DADateTime" class for date handling.
              Example: "cutoff = as_datetime('12/31/2020')"
              Do not use "datetime" directly. Use "today()" instead of "datetime.date.today()".
              To add time to a date variable use .plus(months=...)

            Rules for questions:
            - The question type must be either "yesno" or "date". No other types are permitted.
            - Questions that relate to a specific time period or date must use a "date" field.
            - Do not include unnecessary or duplicate questions.
            - If a legal condition requires checking exceptions (e.g. unless, except), create an explicit question to capture that.
            - Questions must straightforward to answer, if a question is too complex, separate into multiple questions.


            You must output the full interview in valid YAML format.

            Example Input:

            "Check if your family members can get pre-settled status or settled status. If you’re an EU, EEA or Swiss citizen living in the UK,
            some of your family can also apply to come and live in the UK. They can apply for pre-settled or settled status from the EU Settlement
            Scheme if both:
            - you have pre-settled or settled status
            - your relationship with your family member started by 31 December 2020 - unless you’re a Swiss citizen
            If your family member is a child who was born after 31 December 2020, you can also apply for them to come and live in the UK.
            If you came to the UK on a visa after 31 December 2020, you can't use the EU Settlement Scheme to bring your family to the UK.
            You’ll need to check if your visa allows you to bring your family to the UK. The EEA includes EU countries and also Iceland, Liechtenstein
            and Norway."

            Example Code Blocks (Not including questions that should be in the final output):

            ---
            mandatory: True
            code: |
              if (
                is_eu_citizen and
                (is_settled or is_pre_settled) and
                (is_swiss or relation_started_by or was_born_after) and
                not (arrived_on_visa and arrived_after)
              ):
                can_apply = True
              else:
                can_apply = False

              if can_apply:
                can_apply_event
              else:
                cannot_apply_event
            ---
            code: |
              relation_cutoff = as_datetime('12/31/2020')
              if date_of_relation_start <= relation_cutoff:
                relation_started_by = True
              else:
                relation_started_by = False
            ---
            code: |
              birth_cutoff = as_datetime('12/31/2020')
              if date_of_birth > birth_cutoff:
                was_born_after = True
              else:
                was_born_after = False
            ---
            code: |
              arrival_cutoff = as_datetime('12/31/2020')
              if date_of_arrival > arrival_cutoff:
                arrived_after = True
              else:
                arrived_after = False
        """

In [115]:
response = generate_with_reasoner(input_text + skeleton, prompt_2)
reasoning_content = response.choices[0].message.reasoning_content
interview = response.choices[0].message.content

In [116]:
import yaml

lines = interview.splitlines()
cleaned_yaml = "\n".join(lines[1:-1])

with open('interview.yml', 'w') as outfile:
    outfile.write(cleaned_yaml)

In [118]:
prompt_3 = """You are assuming the role of a legal expert, building an expert system. You will be provided a Docassemble
              interview which you need to adjust.

              For each question you must add a 'subquestion' which includes supporting information relevant to the question.

              The existing elements of the file should not be altered (code, questions, events).

              Information should be neatly formatted with headings and bold text.
              This must be written in full explanatory sentences.

              The type of information to include in order of importance:
                - Clarification of uncommon terms in the question.
                - Context to help the user understand what the question is referring to.
                - Guidance to help users interpret how the question applies to their circumstances, relative to their path taken in the interview present.
                - URL links to relevant external resources, if necessary.

              Your output must be the full in interview in YAML format.
            """

In [ ]:
response = generate_with_reasoner(input_text + interview, prompt_3)
reasoning_content = response.choices[0].message.reasoning_content
interview_with_information = response.choices[0].message.content

In [ ]:
lines = interview_with_information.splitlines()
cleaned_yaml = "\n".join(lines[1:-1])

with open('final_interview.yml', 'w') as outfile:
    outfile.write(cleaned_yaml)